In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import time

from math import sqrt
from numpy import loadtxt
from itertools import product
from sklearn import preprocessing
from xgboost import plot_tree
from matplotlib import pyplot

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import  TfidfVectorizer


kernel_with_output = False


In [8]:
kernel_with_output = True
if kernel_with_output:
    sales_train = pd.read_csv('../input/sales_train.csv')
    items = pd.read_csv('../input/items.csv')
    shops = pd.read_csv("../input/shops.csv")
    item_categories = pd.read_csv('../input/item_categories.csv')
    test =pd.read_csv('../input/test.csv')
    sample_submission = pd.read_csv('../input/sample_submission.csv')



In [9]:
sales_train

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0
...,...,...,...,...,...,...
2935844,10.10.2015,33,25,7409,299.00,1.0
2935845,09.10.2015,33,25,7460,299.00,1.0
2935846,14.10.2015,33,25,7459,349.00,1.0
2935847,22.10.2015,33,25,7440,299.00,1.0


In [4]:
if kernel_with_output:
    grid = []
    for block_num in sales_train['date_block_num'].unique():
        cur_shops = sales_train[sales_train['date_block_num']==block_num]['shop_id'].unique()
        cur_items = sales_train[sales_train['date_block_num'] == block_num]['item_id'].unique()
        grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))
    index_cols = ['shop_id', 'item_id', 'date_block_num']
    grid = pd.DataFrame(np.vstack(grid), columns=index_cols, dtype=np.int32)
    sales_train['item_cnt_day'] = sales_train['item_cnt_day'].clip(0,20)
    groups = sales_train.groupby(['shop_id', 'item_id','date_block_num'])
    trainset = groups.agg({'item_cnt_day':'sum', 'item_price':'mean'}).reset_index()

    trainset = trainset.rename(columns={'item_cnt_day':'item_cnt_month'})
    trainset['item_cnt_month'] = trainset['item_cnt_month'].clip(0,20)
    trainset = pd.merge(trainset, items[['item_id','item_category_id']], on='item_id')
    trainset.to_csv('trainset_with_grid.csv')




In [5]:

trainset.head()

,shop_id,item_id,date_block_num,item_cnt_month,item_price,item_category_id
0,0,30,1,20.0,265.0,40
1,1,30,1,10.0,265.0,40
2,2,30,2,1.0,359.0,40
3,2,30,5,1.0,399.0,40
4,2,30,15,1.0,169.0,40


In [6]:
trainset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1609124 entries, 0 to 1609123
Data columns (total 6 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   shop_id           1609124 non-null  int64  
 1   item_id           1609124 non-null  int64  
 2   date_block_num    1609124 non-null  int64  
 3   item_cnt_month    1609124 non-null  float64
 4   item_price        1609124 non-null  float64
 5   item_category_id  1609124 non-null  int64  
dtypes: float64(2), int64(4)
memory usage: 85.9 MB
